In [1]:
!nvidia-smi

Mon May  5 17:30:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8             10W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Package download

!pip install sentencepiece -q
!pip install transformers==4.33 -q
!pip install datasets -q
!pip install peft -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.7 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.16.11 requires transformers>=4.33.1, but you have transformers 4.33.0 which is incompatible.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.33.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you h

In [3]:
# Nllb loading

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "facebook/nllb-200-distilled-600M"
# model_name = "facebook/nllb-200-3.3B" # Larger model

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer.src_lang = "zho_Hant"
tokenizer.tgt_lang = "zho_Hant"
# zho_Hant for Chinese traditional
# eng_Latn for English

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

2025-05-05 17:34:17.082616: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746466457.327872      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746466457.401579      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always 

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location=map

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [7]:
# Load data into dataframes

import pandas as pd

lexicon = pd.read_csv('/kaggle/input/ml2025-bonus/dataset/lexicon_no_en.csv', sep=",", quotechar='"', header=None, encoding="utf-8")
lexicon.columns = ['pyu', 'zho']

lexicon_en = pd.read_csv('/kaggle/input/ml2025-bonus/dataset/lexicon.csv', sep=",", quotechar='"', header=None, encoding="utf-8")
lexicon_en.columns = ['pyu', 'eng', 'zho']

sentences = pd.read_csv('/kaggle/input/ml2025-bonus/dataset/sentences_no_en.csv', sep=",", quotechar='"', header=None, encoding="utf-8")
sentences.columns = ['pyu', 'zho']

sentences_en = pd.read_csv('/kaggle/input/ml2025-bonus/dataset/sentences.csv', sep=",", quotechar='"', header=None, encoding="utf-8")
sentences_en.columns = ['pyu', 'eng', 'zho']

#lexicon.sample(5)
#lexicon_en.sample(10)
#sentences.sample(5)
#sentences_en.sample(10)

In [20]:
# Testing the performances of original tokenization

import re

def word_tokenize(text):
    """
    Split a text into words, numbers, and punctuation marks
    (for languages where words are separated by spaces)
    """
    return re.findall('(\w+|[^\w\s])', text)

def df_tokenize(df):

    df['pyu_toks'] = df.pyu.apply(tokenizer.tokenize)
    df['zho_toks'] = df.zho.apply(tokenizer.tokenize)
    df['pyu_words'] = df.pyu.apply(word_tokenize)
    df['zho_words'] = df.zho.apply(word_tokenize)
    
    return df

def cal_tokperword(df):

    stats = df[['pyu_toks', 'zho_toks', 'pyu_words', 'zho_words']].map(len).describe()
    print(stats.pyu_toks['mean'] / stats.pyu_words['mean'])
    print(stats.zho_toks['mean'] / stats.zho_words['mean'])

    return stats

def check_unk(df, column):

    texts_with_unk = [
        text for text in df[column]
        if tokenizer.unk_token_id in tokenizer(text).input_ids
    ]
    print(len(texts_with_unk))

lexicon = df_tokenize(lexicon)
lexicon_en = df_tokenize(lexicon_en)
sentences = df_tokenize(sentences)
sentences_en = df_tokenize(sentences_en)

print("toks per word of lexicon:")
stats_lexicon = cal_tokperword(lexicon)
print("toks per word of lexicon_en:")
stats_lexicon = cal_tokperword(lexicon_en)
print("toks per word of sentences:")
stats_sentences = cal_tokperword(sentences)
print("toks per word of sentences_en:")
stats_sentences = cal_tokperword(sentences_en)

print("total unk in lexicon zho:")
check_unk(lexicon, "zho")
print("total unk in lexicon pyu:")
check_unk(lexicon, "pyu")
print("total unk in lexicon_en zho:")
check_unk(lexicon_en, "zho")
print("total unk in lexicon_en pyu:")
check_unk(lexicon_en, "pyu")
print("total unk in sentences zho:")
check_unk(sentences, "zho")
print("total unk in sentences pyu:")
check_unk(sentences, "pyu")
print("total unk in sentences_en zho:")
check_unk(sentences_en, "zho")
print("total unk in sentences pyu:")
check_unk(sentences_en, "pyu")

#show datas
#lexicon.sample(10)
#sentences.sample(10)
#stats_lexicon
#stats_sentences

toks per word of lexicon:
2.2840203274985886
2.3545659526493803
toks per word of lexicon_en:
1.738603297769156
2.2698515171078117
toks per word of sentences:
1.5806060606060606
3.925910765452312
toks per word of sentences_en:
1.5050038491147038
3.165337423312884
total unk in lexicon zho:
97
total unk in lexicon pyu:
0
total unk in lexicon_en zho:
203
total unk in lexicon_en pyu:
0
total unk in sentences zho:
223
total unk in sentences pyu:
36
total unk in sentences_en zho:
142
total unk in sentences pyu:
14


In [32]:
# Add missing tokens

import pandas as pd
from tqdm.auto import tqdm
import re
from collections import Counter
import sentencepiece as spm
from datasets import load_dataset


all_texts_zh = lexicon['zho'].dropna().tolist() + sentences['zho'].dropna().tolist() + lexicon_en['zho'].dropna().tolist() + sentences_en['zho'].dropna().tolist()

all_texts_file_zh = 'zho_texts_plain.txt'
with open(all_texts_file_zh, 'w', encoding='utf-8') as f:
    for text in all_texts_zh:
        print(text, file=f)


required_chars_zh = set()

for text in tqdm(all_texts_zh):
    for char in text:
        tokens = tokenizer.tokenize(char)
        if tokens == ['▁', '<unk>']:
            required_chars_zh.add(char)

required_chars_zh_str = "".join(sorted(list(required_chars_zh)))
print(f"需要強制包含的中文單字元: {required_chars_zh_str}")

spm.SentencePieceTrainer.train(
    input=all_texts_file_zh,
    model_prefix='spm_zho_new',
    vocab_size=5800,
    character_coverage=1,
    num_threads=16,
    train_extremely_large_corpus=False,
    add_dummy_prefix=False,
    max_sentencepiece_length=128,
    max_sentence_length=4192 * 4,
    pad_id=0,
    eos_id=1,
    unk_id=2,
    bos_id=-1,
    required_chars=required_chars_zh_str,
)

#工作進度至此

from sentencepiece import sentencepiece_model_pb2 as sp_pb2_model
from transformers import NllbTokenizer

# 載入 NLLB tokenizer
model_name = 'facebook/nllb-200-distilled-600M'
tokenizer_nllb = NllbTokenizer.from_pretrained(model_name)

# 載入你訓練的中文 SentencePiece 模型
sp_trained_zh = spm.SentencePieceProcessor(model_file='spm_zho_new.model')
added_spm_zh = sp_pb2_model.ModelProto()
added_spm_zh.ParseFromString(sp_trained_zh.serialized_model_proto())
old_spm_nllb = sp_pb2_model.ModelProto()
old_spm_nllb.ParseFromString(tokenizer_nllb.sp_model.serialized_model_proto())

# 將新的中文 tokens 添加到 NLLB 的 SPM 中
nllb_tokens_set = {p.piece for p in old_spm_nllb.pieces}
prev_min_score = old_spm_nllb.pieces[-1].score
for p in added_spm_zh.pieces:
    piece = p.piece
    if p.type != 1:  # 排除特殊 token
        continue
    if piece not in nllb_tokens_set:
        new_p = sp_pb2_model.ModelProto().SentencePiece()
        new_p.piece = piece
        new_p.score = p.score + prev_min_score
        old_spm_nllb.pieces.append(new_p)

# 保存新的 NLLB SPM 模型
NEW_SPM_NAME_ZH = 'spm_nllb_zho_extended_268k.model'
with open(NEW_SPM_NAME_ZH, 'wb') as f:
    f.write(old_spm_nllb.SerializeToString())

# 載入使用新 SPM 模型的 tokenizer
tokenizer = NllbTokenizer.from_pretrained(model_name, vocab_file='spm_zho_new.model')
print(len(tokenizer_nllb), len(tokenizer))
added_vocab_zh = set(tokenizer.get_vocab()).difference(set(tokenizer_nllb.get_vocab()))
print(added_vocab_zh)

  0%|          | 0/4385 [00:00<?, ?it/s]

需要強制包含的中文單字元: 
256204 6004
{'美好', '我的媽媽', '哎', '到田裡', '古時候', '住的', '發怒', '粥', '路上要', '和我', '一下子', '的傳說', '郵局', '等一下要比賽', '奶奶煮', '過了', '不好', '舞會時', '果實', '客人', '歡迎', '我說', '桌子', '小學', '要吃', '珍惜', '我們想', '我的名字', '器具', '泥土', '外祖母', '遠方', '我弟弟', '給你們', '的兄弟', '我要回去', '的也有', '祝賀', '飼養', '總是把', '越來越多', '傳統除草祭', '帶回', '原住民語', '濁', '板凳', '貓頭鷹', '殷', '有女', '撫慰', '依靠', '由祭司', '嫂', '地瓜葉跟龍鬚菜', '很有趣', '族語的', '十四', '他的名字是', '要用', '石碑', '那裡有', '你是怎麼學', '大聲一點', '明天', '得我頭好', '這兩', '樑', '鹿角上', '可以讓', '獨自', '獻祭', '叮嚀', '先去', '一百', '夏天', '屎', '套索', '巫術', '一個人', '算是', '哪裡', '樂器', '物和食', '喉嚨', '搶奪', '責罵', '濕的', '創作了', '照相', '喂', '的名字', '編花環', '買了', '田鼠', '的孩子', '琉', '個小孩', '人像', '傳承', '便當', '廚房', '從此', '地叫', '釀酒', '聯絡處', '母輩', '相互幫忙', '的時間', '怡然', '我要這', '搭火車', '說族語', '地形', '非常優美', '請問', '慶典', '一手持', '麻煩', '了太陽', '獵場', '仟', '我們可以一起', '的變', '整個', '初鹿部落', '之處', '帶的', '撐', '釋迦', '我們部落的', '在回', '將我們的', '還沒', '很高', '的傳說故事', '青少年', '惱', '的外套', '要去', '其在', '弟弟', '我們要', '後天', '是幾', '龍過脈', '於是他們', '好像', '大家好', '會議'

In [40]:
check_unk(lexicon, 'pyu')

234


In [10]:
# Add new language tag

import json
import os
import shutil
from typing import List, Tuple

from transformers import NllbTokenizer
from transformers.models.nllb.tokenization_nllb import FAIRSEQ_LANGUAGE_CODES


def update_nllb_tokenizer(
    old_tokenizer: NllbTokenizer,
    new_spm_path: str,
    new_lang_codes: List[str],
) -> NllbTokenizer:
    """
    Create a new tokenizer for NLLB, with an updated sentencepiece model and some new language codes.
    In order to get rid of the old (and wrong) added token encoders/decoders, we save the tokenizer to disk and remove those files.
    :param old_tokenizer: the original tokenizer
    :param new_spm_path: path to the file with the sentncepiece model
    :param new_lang_codes: list of the new codes to add to the tokenizer
    :return:
    """
    TKN_DIR = "old_tokenizer"  # todo: make it a temp dir
    old_tokenizer.save_pretrained(TKN_DIR)

    with open(f"{TKN_DIR}/tokenizer_config.json", "r") as f:
        cfg = json.load(f)
    cfg["added_tokens_decoder"] = {
        k: v
        for k, v in cfg["added_tokens_decoder"].items()
        if k in ["0", "1", "2", "3"]
    }
    cfg["additional_special_tokens"] = []
    with open(f"{TKN_DIR}/tokenizer_config.json", "w") as f:
        json.dump(cfg, f, indent=2)

    # this contains added tokens: language codes and mask
    os.remove(f"{TKN_DIR}/added_tokens.json")
    os.remove(f"{TKN_DIR}/special_tokens_map.json")
    os.remove(f"{TKN_DIR}/sentencepiece.bpe.model")
    shutil.copy(new_spm_path, f"{TKN_DIR}/sentencepiece.bpe.model")

    new_tokenizer = NllbTokenizer.from_pretrained(
        TKN_DIR,
        additional_special_tokens=sorted(FAIRSEQ_LANGUAGE_CODES + new_lang_codes),
    )
    return new_tokenizer

from transformers import AutoTokenizer, NllbTokenizer

# 你的新的 SentencePiece 模型路徑
new_spm_path = "/path/to/your/new_spm.model"

# 你想要添加的新語言代碼列表
new_lang_codes = ["pyu_Latn"]

# 更新 tokenizer
new_tokenizer = update_nllb_tokenizer(tokenizer, new_spm_path, new_lang_codes)

# 現在你可以使用 new_tokenizer 了
print(f"新的特殊 token: {new_tokenizer.additional_special_tokens}")
print(f"卑南語的 token ID: {new_tokenizer.convert_tokens_to_ids('pyu_Latn')}")

AttributeError: 'NllbTokenizerFast' object has no attribute 'added_tokens_encoder'

In [ ]:
# Training data preparation

source_lang = "pyu"
target_lang = "zho"

def preprocess_function(examples):
    inputs = examples[source_lang]
    targets = examples[target_lang]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True, src_lang=source_lang, tgt_lang=target_lang)
    return model_inputs

tokenized_lexicon = dataset_lexicon.map(preprocess_function, batched=True)
tokenized_sentences = dataset_sentences.map(preprocess_function, batched=True)

# 合併兩個 datasets 作為訓練資料
from datasets import concatenate_datasets
train_dataset = concatenate_datasets([tokenized_lexicon, tokenized_sentences])
train_dataset = train_dataset.remove_columns(dataset_lexicon.column_names) # 移除原始文本欄位
train_dataset = train_dataset.rename_column("zho", "labels") # 將目標語言欄位命名為 "labels"

In [9]:
# Training the model

from peft import LoraConfig, TaskType, get_peft_model
from transformers import TrainingArguments, DataCollatorForSeq2Seq, Trainer

lora_config = LoraConfig(
    r=8,                     
    lora_alpha=16,             
    lora_dropout=0.15,          
    bias="none",                  
    task_type=TaskType.SEQ_2_SEQ_LM 
)
lora_model = get_peft_model(model, lora_config)

training_args = TrainingArguments(
    output_dir="./nllb-lora-puyuma",
    per_device_train_batch_size=2,     
    gradient_accumulation_steps=4, 
    learning_rate=1e-4,            
    num_train_epochs=5,              
    fp16=True,                          
    evaluation_strategy="epoch",       
    save_strategy="epoch",            
    logging_dir="./logs",                 
    report_to="none"
)


data_collator = DataCollatorForSeq2Seq(tokenizer, model=lora_model)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

In [ ]:
# 評估 (如果有驗證集)
# trainer.evaluate()

# 推論
def translate(text, model, tokenizer, src_lang="pyu", tgt_lang="zho"):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", src_lang=src_lang, tgt_lang=tgt_lang).to(model.device)
    generated_tokens = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang],
        max_length=128,
        num_beams=5,
        repetition_penalty=1.5,
        length_penalty=1.0,
        early_stopping=True
    )
    translation = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    return translation

# 測試翻譯
test_text = "你好嗎" # 假設這是卑南語
translated_text = translate(test_text, lora_model, tokenizer, src_lang="pyu", tgt_lang="zho")
print(f"卑南語: {test_text}")
print(f"中文翻譯: {translated_text}")